In [1]:
from Lib.myModules.my_ML_Modules_1 import *
import math
import sys
import contextlib

In [2]:
data_dir = "../data"
infantNum = 7
ECG_dataset = f"{data_dir}/infant{infantNum}_ecg"
RESP_dataset = f"{data_dir}/infant{infantNum}_resp"
# Read all the data
signal_ecg_0 = wfdb.rdsamp(f"{data_dir}/infant{infantNum}_ecg")
signal_resp_0 = wfdb.rdsamp(f"{data_dir}/infant{infantNum}_resp")
print(f'ECG DATA: {signal_ecg_0[1]}')
print(F'RESP DATA: {signal_resp_0[1]}')
totalDuration_ECG = num2Time(num=signal_ecg_0[1]['sig_len'], fs=signal_ecg_0[1]['fs'])
print(f'{totalDuration_ECG} sec, {secs2minutes(totalDuration_ECG)} minutes, {secs2hours(totalDuration_ECG)} hours')
totalDuration_RESP = num2Time(num=signal_resp_0[1]['sig_len'], fs=signal_resp_0[1]['fs'])
print(f'{totalDuration_RESP} sec, {secs2minutes(totalDuration_RESP)} minutes, {secs2hours(totalDuration_RESP)} hours')

ECG DATA: {'fs': 500, 'sig_len': 36604500, 'n_sig': 1, 'base_date': None, 'base_time': None, 'units': ['mV'], 'sig_name': ['II'], 'comments': []}
RESP DATA: {'fs': 50, 'sig_len': 3660288, 'n_sig': 1, 'base_date': None, 'base_time': None, 'units': ['NU'], 'sig_name': ['RESP'], 'comments': []}
73209.0 sec, 1220.15 minutes, 20.335833333333333 hours
73205.76 sec, 1220.096 minutes, 20.334933333333332 hours


In [3]:
# initialize the training
start_time_secs = hours2secs(1)
end_time_secs = hours2secs(1.1)
lr_model, pr_model, svr_rbf_model, svr_lin_model, svr_poly_model = training_pipeline(start_time_secs=start_time_secs, 
                                                                                     end_time_secs=end_time_secs, 
                                                                                     signal_ecg_0=signal_ecg_0,
                                                                                     signal_resp_0=signal_resp_0,
                                                                                     ECG_dataset=ECG_dataset,
                                                                                     RESP_dataset=RESP_dataset)

start: 1800000, end: 1980000
start: 180000, end: 198000
90th percentile: 0.13163502856402232, 10th percentile: -0.19394746812095595, IQR: 0.3255824966849783
Identified outliers: 4384
Data Shape: (180000,)
90th percentile: 16.799460791688748, 10th percentile: 13.103400356137513, IQR: 3.696060435551235
Identified outliers: 40
Data Shape: (18000,)
[[148.23132673   0.88265237]
 [  0.88265237 473.510453  ]]
Pearsons correlation: 0.003
Spearmans correlation: -0.003
---------------------------------------------
--- Linear Regression ---
Coefficient of determination: 3.7373550835217983e-06
Intercept: [158.08038]
Coefficients: [[-0.00344777]]
Mean absolute error: 13.82
Mean squared error: 472.49
Root mean squared error: 21.74
---------------------------------------------
--- Polynomial Regression ---
Coefficient of determination: 0.004533169777868218
Intercept: [160.49753]
Coefficients: [[-2.7260267e-09 -1.3486317e-07 -2.0376071e-06 -2.9506271e-05
   1.1621796e-06 -1.1443376e-08]]
Mean absolute

In [4]:
step = 0.25
logFilePath = f"../logs/infant{infantNum}_train_output_step{step}.log"

with open(logFilePath, 'a') as f:
    with contextlib.redirect_stdout(f):
        for i in np.arange(10, 20, step):
            print("-------------------------------------------------------------------------")
            print(f"Data bin index: {i:.1f}")
            start_time_secs = hours2secs(i)
            end_time_secs = hours2secs(i+step)
            lr_model, pr_model, svr_rbf_model, svr_lin_model, svr_poly_model = cascade_training_pipeline(lr_model=lr_model, 
                                                                                                         pr_model=pr_model, 
                                                                                                         svr_rbf_model=svr_rbf_model, 
                                                                                                         svr_lin_model=svr_lin_model, 
                                                                                                         svr_poly_model=svr_poly_model, 
                                                                                                         start_time_secs=start_time_secs, 
                                                                                                         end_time_secs=end_time_secs,
                                                                                                         signal_ecg_0=signal_ecg_0,
                                                                                                         signal_resp_0=signal_resp_0,
                                                                                                         ECG_dataset=ECG_dataset,
                                                                                                         RESP_dataset=RESP_dataset)

In [5]:
# save the model
for i in [lr_model, pr_model, svr_rbf_model, svr_lin_model, svr_poly_model]:
    filename = f"../models/infant_{infantNum}/{i}_step{step}.sav"
    joblib.dump(i, filename)
